In [56]:
import tensorflow
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [57]:
##load the ann train model, scaler pickel file and ohe pickel
model=load_model('model.h5')


## load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo=pickle.load(file)

with open('lable_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)


In [58]:
##example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 5000
}

In [59]:
input_df = pd.DataFrame([input_data])

In [61]:
print(input_df.columns)

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')


In [62]:
geo_encoded = label_encoder_geo.transform(input_df[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [63]:
#one hot encode geography column
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\lenovo\Desktop\prajwal\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [64]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,5000


In [65]:
##encode categorical variable
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,5000


In [66]:
##concatination on one hot encoded
input_df= pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,5000,1.0,0.0,0.0


In [67]:
##scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -1.65923237,  1.00150113,
        -0.57946723, -0.57638802]])

In [68]:
##predict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 1s 867ms/step


array([[0.02361661]], dtype=float32)

In [69]:
prediction_proba = prediction[0][0]
prediction_proba

0.023616606

In [70]:
if prediction_proba > 0.5:
    print('customer is likely to churn.')
else:
    print('customer is not likely to churn')

customer is not likely to churn
